In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# load prices data
sp500 = pd.read_csv("../prices/GSPC.csv",index_col=0,parse_dates=True)
nasdaq = pd.read_csv("../prices/NDX.csv",index_col=0,parse_dates=True)
eurostoxx = pd.read_csv("../prices/STOXX50E.csv",index_col=0,parse_dates=True)
nikkei = pd.read_csv("../prices/N225.csv",index_col=0,parse_dates=True)
#remove duplicates keep last one
sp500 = sp500[~sp500.index.duplicated(keep='last')]
nasdaq = nasdaq[~nasdaq.index.duplicated(keep='last')]
eurostoxx = eurostoxx[~eurostoxx.index.duplicated(keep='last')]
nikkei = nikkei[~nikkei.index.duplicated(keep='last')]

In [3]:
# make union between indexes
index_union = sp500.index.union(eurostoxx.index.union(nikkei.index))
open_close_prices = pd.DataFrame(index=index_union)
open_close_prices.loc[sp500.index,'sp500_open'] = sp500.loc[:,'Open']
open_close_prices.loc[sp500.index,'sp500_close'] = sp500.loc[:,'Close']
open_close_prices.loc[sp500.index,'nasdaq_open'] = nasdaq.loc[:,'Open']
open_close_prices.loc[sp500.index,'nasdaq_close'] = nasdaq.loc[:,'Close']
# intersection between american and european + japanese opening days
open_close_prices.loc[eurostoxx.index,'euro_open'] = eurostoxx.loc[:,'Open']
open_close_prices.loc[eurostoxx.index,'euro_close'] = eurostoxx.loc[:,'Close']
open_close_prices.loc[nikkei.index,'nikkei_open'] = nikkei.loc[:,'Open']
open_close_prices.loc[nikkei.index,'nikkei_close'] = nikkei.loc[:,'Close']

In [4]:
# fill nan values with open=close (no return)
open_close_prices['sp500_close'].fillna(method='ffill',inplace=True)
open_close_prices['nasdaq_close'].fillna(method='ffill',inplace=True)
open_close_prices['nikkei_close'].fillna(method='ffill',inplace=True)
open_close_prices['euro_close'].fillna(method='ffill',inplace=True)
open_close_prices['sp500_open'].fillna(open_close_prices['sp500_close'],inplace=True)
open_close_prices['nasdaq_open'].fillna(open_close_prices['nasdaq_close'],inplace=True)
open_close_prices['nikkei_open'].fillna(open_close_prices['nikkei_close'],inplace=True)
open_close_prices['euro_open'].fillna(open_close_prices['euro_close'],inplace=True)

In [5]:
open_close_prices

,sp500_open,sp500_close,nasdaq_open,nasdaq_close,euro_open,euro_close,nikkei_open,nikkei_close
Date,,,,,,,,
2013-01-02,1448.992997,1456.241781,2724.067637,2738.764463,2677.155794,2699.569115,NaN,NaN
2013-01-03,1456.140648,1453.362359,2739.034281,2726.426141,2692.752434,2695.040771,NaN,NaN
2013-01-04,1456.017477,1460.564153,2726.875714,2718.097261,2687.471860,2698.852793,10750.016571,10672.173913
2013-01-07,1455.041312,1456.003900,2709.639078,2717.786440,2704.208855,2686.213649,10740.000000,10605.300000
2013-01-08,1454.147965,1451.561604,2719.258148,2712.934659,2678.588291,2686.754133,10541.425293,10501.358025
...,...,...,...,...,...,...,...,...
2020-06-24,3097.062500,3039.562500,10169.875000,9990.312500,3259.750000,3178.250000,22497.500000,22477.500000
2020-06-25,3033.250000,3072.500000,10000.500000,10088.062500,3168.250000,3208.750000,22222.500000,22220.000000
2020-06-26,3061.000000,3000.000000,10075.312500,9843.875000,3230.750000,3182.750000,22405.000000,22452.500000


In [11]:
close_prices = pd.DataFrame(index = open_close_prices.index)
close_prices['N225'] = open_close_prices['nikkei_close']
close_prices['STOXX50E'] = open_close_prices['euro_close']
close_prices['NDX'] = open_close_prices['nasdaq_close']
close_prices['GSPC'] = open_close_prices['sp500_close']
close_prices.fillna(method='bfill',inplace=True)
close_prices.to_csv('../closing_prices.csv')

In [6]:
daily_returns = pd.DataFrame(index=open_close_prices.index)
daily_returns['N225'] = \
(open_close_prices['nikkei_close']-open_close_prices['nikkei_open'])/open_close_prices['nikkei_open']
daily_returns['STOXX50E'] = \
(open_close_prices['euro_close']-open_close_prices['euro_open'])/open_close_prices['euro_open']
daily_returns['NDX'] = \
(open_close_prices['nasdaq_close']-open_close_prices['nasdaq_open'])/open_close_prices['nasdaq_open']
daily_returns['GSPC'] = \
(open_close_prices['sp500_close']-open_close_prices['sp500_open'])/open_close_prices['sp500_open']
daily_returns = daily_returns.fillna(0)

In [8]:
daily_returns.to_csv('../daily_returns.csv')